In [1]:
import pandas as pd
import pd
from vega_datasets import data
import altair as alt
import matplotlib.pyplot as plt

In [2]:
alc_df = pd.read_csv('filtered_alc_mort.csv')
alc_df.head()
alc_df["Fatalities"] = alc_df["Data_Value"].replace("-" , 0).fillna(0).astype(int)

FileNotFoundError: [Errno 2] No such file or directory: 'filtered_alc_mort.csv'

In [ ]:
gb = alc_df.groupby(['LocationDesc', "Cause_of_Death", "Sex"]).size()
gb

In [ ]:
gb = alc_df.groupby(['LocationDesc', 'ConditionType']).count().reset_index()

#counts = gb.size().to_frame(name='counts')

gb = gb.iloc[:,0:3]
gb

In [ ]:
counts_p = gb.pivot(index='LocationDesc', columns='ConditionType')
counts_p.head()

In [ ]:
import geopandas
state = geopandas.read_file("ne_110m_admin_1_states_provinces/ne_110m_admin_1_states_provinces.shp")
state.head()

In [ ]:
full = pd.merge(state, alc_df, left_on='woe_name', right_on='LocationDesc', how='outer')
full.head()
full.shape

In [ ]:
states = alt.topo_feature(data.us_10m.url, feature='states')
states

In [ ]:
alc_df.head()

In [ ]:
alc_df_age = alc_df.groupby(by = ['LocationDesc', 'AgeGroup'], as_index= False).sum("Fatalities")

alc_df_age = alc_df_age.iloc[:, [0,1,4]]
alc_df_age = alc_df_age[alc_df_age["AgeGroup"] != "Under 21"]
alc_df_age

In [ ]:
full = pd.merge(state, alc_df_age, left_on='woe_name', right_on='LocationDesc', how='outer')
full.head()
full.shape

In [ ]:
import altair as alt
from vega_datasets import data
import geopandas as gpd

ages = full['AgeGroup'].unique()

input_dropdown = alt.binding_select(options=['Europe','Japan','USA'], name='Region ')
selection = alt.selection_point(fields=['Origin'], bind=input_dropdown)
color = alt.condition(
    selection,
    alt.Color('Origin:N').legend(None),
    alt.value('lightgray')
)

# define a pointer selection
click_state = alt.selection_point(fields=["LocationDesc"])

# create a choropleth map using a lookup transform
# define a condition on the opacity encoding depending on the selection
choropleth = (
    alt.Chart(full)
    .mark_geoshape()
    .transform_lookup(
        lookup="id", from_=alt.LookupData(us_population, "id", ["population", "state"])
    )
    .encode(
        color="population:Q",
        opacity=alt.condition(click_state, alt.value(1), alt.value(0.2)),
        tooltip=["state:N", "population:Q"],
    )
    .project(type="albersUsa")
)

# create a bar chart with a similar condition on the opacity encoding.
bars = (
    alt.Chart(
        us_population.nlargest(15, "population"), title="Top 15 states by population"
    )
    .mark_bar()
    .encode(
        x="population",
        opacity=alt.condition(click_state, alt.value(1), alt.value(0.2)),
        color="population",
        y=alt.Y("state").sort("-x"),
    )
)

(choropleth & bars).add_params(click_state)

In [ ]:
full['geometry'] = full.geometry

In [ ]:
ages = full['AgeGroup'].unique()
ages = list(set(ages))
ages

In [ ]:
# List of all unique Measures
measures = ages

# Create drop-down, using the "measures" list
input_dropdown = alt.binding_select(options=measures, name='Age Groups')

# Creating a binding "selection" object in Vega
# which is mapped filter the "Measure" field in the data
selection = alt.selection_point(fields=['Age Groups'], bind=input_dropdown)

state_fig = (
    alt.Chart(full)   # Note I removed the .filter()
    .mark_geoshape(strokeWidth=1, stroke="darkgreen")
    .encode(
        shape="geometry:G",
        color="Value",
        tooltip=['LocationDesc', 'AgeGroup', "Fatalities"],
    )
    .transform_lookup(
        lookup="geometry", 
        from_=alt.LookupData(data=counties, key="id"), 
        as_="geometry"
    )
    .transform_filter(   # transform_filter() allows dynamic selection
        selection  
    )
    .add_params(   # add_params() make "selection" variable available to chart
        selection
    )
    .project(type="albersUsa")
    .properties(width=400)
)
state_fig

In [ ]:
counties = alt.topo_feature(data.us_10m.url, 'state')

In [ ]:
import polars as pl
alt.data_transformers.disable_max_rows()

state = alt.topo_feature(data.us_10m.url, 'state')

# List of all unique Measures
measures = ages

# Create drop-down, using the "measures" list
input_dropdown = alt.binding_select(options=measures, name='Measure')

# Creating a binding "selection" object in Vega
# which is mapped filter the "Measure" field in the data
selection = alt.selection_point(fields=['Measure'], bind=input_dropdown)

state_fig = (
    alt.Chart(full.filter("AgeGroup")=="Overall")   # Note I removed the .filter()
    .mark_geoshape(strokeWidth=1, stroke="darkgreen")
    .encode(
        shape="geometry:G",
        color="Fatalities",
        tooltip=['LocationDesc', 'AgeGroup', "Fatalities"]
    ).add_params(   # add_params() make "selection" variable available to chart
        selection
    )
    .project(type="albersUsa")
    .properties(width=400)
)
state_fig

In [ ]:
    ).transform_filter(   # transform_filter() allows dynamic selection
        selection  
    )


state_fig = (
    alt.Chart(full.filter(pl.col("Measure")=="Total Population"))
    .mark_geoshape(strokeWidth=1, stroke="darkgreen")
    .encode(
        shape="geo:G",
        color="Value",
        tooltip=["County Name", "GEOID", "Measure", "Value"],
    )
    .transform_lookup(
        lookup="GEOID", 
        from_=alt.LookupData(data=counties, key="id"), 
        as_="geo"
    )
    .project(type="albersUsa")
    .properties(width=400)
)
state_fig

In [ ]:
import altair as alt
from vega_datasets import data

# Load the US state data
states = alt.topo_feature(data.us_10m.url, 'states')

# Create a dropdown selection
category_dropdown = alt.binding_select(options=['Population', 'Unemployment Rate'])
selection = alt.selection_single(fields=['category'], bind=category_dropdown, name='Select')

# Define the base map
base = alt.Chart(states).mark_geoshape(
    fill='#EEE',
    stroke='white'
).properties(
    width=500,
    height=300
).project('albersUsa')

# Define the choropleth map
choropleth = alt.Chart(states).mark_geoshape().encode(
        color=alt.condition(
            selection,
            alt.Color('properties.category:N', scale=alt.Scale(scheme='blues')),
            alt.value('lightgray')
        ),
        tooltip=[
            alt.Tooltip('properties.name:N', title='State'),
            alt.Tooltip('properties.category:Q', title='Category Value')
        ]
    ).transform_lookup(
        lookup='id',
        from_=alt.LookupData(data=states, key='id', fields=['category'])
    ).add_params(
        selection
    ).properties(
        title="US Chloropleth Map"
    )

# Combine the base map and the choropleth map
us_choropleth = base + choropleth

# Show the map
us_choropleth

In [ ]:
import pandas as pd
pop_df = pd.read_csv('nst-est2019-01.csv')
pop_df['state'] = list(map(lambda x: x.replace('.', ''), pop_df['state']))

In [6]:
import pandas as pd 
df = pd.read_csv('../filtered_alc_mort.csv')
df

,Unnamed: 0,LocationDesc,ConditionType,Category,Cause_of_Death,Data_Value,Data_Value_Alt,Data_Value_Footnote_Symbol,Data_Value_Footnote,Effect,ConsumptionPattern,Sex,AgeCategory,AgeGroup,ConditionTypeID,AgeCategoryID
0,1,West Virginia,Chronic,Total,Total for All Causes,0,0.0,NaN,NaN,Harmful Effects,Excessive Alcohol Use,Female,Under 21,Overall,EXCSVE,UN21
1,2,West Virginia,Total,Total,Total for All Causes,7,7.0,NaN,NaN,Harmful Effects,Excessive Alcohol Use,Female,Under 21,Overall,EXCSVE,UN21
2,3,United States,Acute,Alcohol-Related Poisonings,Alcohol poisoning,13,13.0,NaN,NaN,Harmful Effects,Excessive Alcohol Use,Female,Under 21,Under 21,ACUTE,UN21
3,4,United States,Acute,Alcohol-Related Poisonings,Alcohol poisoning,1761,1761.0,NaN,NaN,Harmful Effects,Excessive Alcohol Use,Male,Other,Overall,ACUTE,OTH
4,5,West Virginia,Acute,Total,Total for All Causes,367,367.0,NaN,NaN,Harmful Effects,Any Alcohol Use,Male,Other,Overall,ANYALC,OTH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62499,62500,Wyoming,Chronic,Total,Total for All Causes,83,83.0,NaN,NaN,Harmful Effects,Any Alcohol Use,Overall,Other,65+,ANYALC,OTH
62500,62501,Wyoming,Total,Total,Total for All Causes,102,102.0,NaN,NaN,Harmful Effects,Any Alcohol Use,Overall,Other,65+,ANYALC,OTH
62501,62502,Wyoming,Acute,Total,Total for All Causes,19,19.0,NaN,NaN,Harmful Effects,Excessive Alcohol Use,Overall,Other,65+,EXCSVE,OTH
62502,62503,Wyoming,Chronic,Total,Total for All Causes,77,77.0,NaN,NaN,Harmful Effects,Excessive Alcohol Use,Overall,Other,65+,EXCSVE,OTH


NameError: name 'full' is not defined